In [2]:
# data normalisation with sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams ["figure.figsize"] = 10,6
import seaborn as sns
from scipy import stats
import math
import datetime
import urllib
import zipfile
import statsmodels.tsa.stattools as sts
from statsmodels.tsa.seasonal import seasonal_decompose
import plotly.express as px
import glob
ohe = OneHotEncoder
le = LabelEncoder()
from sklearn.compose import ColumnTransformer
le = LabelEncoder()

from sklearn.model_selection import train_test_split

from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression as LR

from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import cohen_kappa_score

In [3]:
bike_station = pd.read_csv('stations.csv')


In [4]:
bike_station.tail(5)

,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type
1226762,300,2011-12-31 23:41:19,2011-12-31 23:46:20,31201,15th & P St NW,31214,17th & Corcoran St NW,W01459,Member
1226763,387,2011-12-31 23:46:43,2011-12-31 23:53:10,31223,Convention Center / 7th & M St NW,31201,15th & P St NW,W01262,Member
1226764,261,2011-12-31 23:47:27,2011-12-31 23:51:49,31107,Lamont & Mt Pleasant NW,31602,Park Rd & Holmead Pl NW,W00998,Member
1226765,2060,2011-12-31 23:55:12,2012-01-01 00:29:33,31205,21st & I St NW,31222,New York Ave & 15th St NW,W00042,Member
1226766,468,2011-12-31 23:55:56,2012-01-01 00:03:45,31221,18th & M St NW,31111,10th & U St NW,W01319,Member


In [5]:
#FIX THE FRAME

#remove some uneccesary columns, change Duration to minutes, change members to two categories, make the categories workable. 

In [6]:
bike_station2 = bike_station.drop(['Start station', 'End station', 'Bike number'], axis=1)
bike_station2

,Duration,Start date,End date,Start station number,End station number,Member type
0,3548,2011-01-01 00:01:29,2011-01-01 01:00:37,31620,31620,Member
1,346,2011-01-01 00:02:46,2011-01-01 00:08:32,31105,31101,Casual
2,562,2011-01-01 00:06:13,2011-01-01 00:15:36,31400,31104,Member
3,434,2011-01-01 00:09:21,2011-01-01 00:16:36,31111,31503,Member
4,233,2011-01-01 00:28:26,2011-01-01 00:32:19,31104,31106,Casual
...,...,...,...,...,...,...
1226762,300,2011-12-31 23:41:19,2011-12-31 23:46:20,31201,31214,Member
1226763,387,2011-12-31 23:46:43,2011-12-31 23:53:10,31223,31201,Member
1226764,261,2011-12-31 23:47:27,2011-12-31 23:51:49,31107,31602,Member
1226765,2060,2011-12-31 23:55:12,2012-01-01 00:29:33,31205,31222,Member


In [7]:
#most used stations?

#31200 for both start end end

In [8]:
bike_station2.dtypes

Duration                 int64
Start date              object
End date                object
Start station number     int64
End station number       int64
Member type             object
dtype: object

In [9]:
bike_station2["Start date"] = bike_station2["Start date"].astype('datetime64[ns]')

In [10]:
bike_station2["End date"] = bike_station2["End date"].astype('datetime64[ns]')

In [11]:
bike_station2["Duration"] = bike_station2["Duration"].astype('int64')

In [12]:
bike_station2.dtypes

Duration                         int64
Start date              datetime64[ns]
End date                datetime64[ns]
Start station number             int64
End station number               int64
Member type                     object
dtype: object

In [13]:
#bike_station2["Start date"]

In [14]:
minute_frame = (bike_station2["Duration"]) / (60)
#lst = []
#for each in minute_frame:
#    lst.append(str(each).split('.')[0])

#final_minute = [int(i) for i in lst]

#final_minute, done shows Duration in minutes! need to change to Int

In [15]:
bike_station2['dura_min']=(bike_station2['Duration']/60).round(0)

In [16]:
bike_station2.head()

,Duration,Start date,End date,Start station number,End station number,Member type,dura_min
0,3548,2011-01-01 00:01:29,2011-01-01 01:00:37,31620,31620,Member,59.0
1,346,2011-01-01 00:02:46,2011-01-01 00:08:32,31105,31101,Casual,6.0
2,562,2011-01-01 00:06:13,2011-01-01 00:15:36,31400,31104,Member,9.0
3,434,2011-01-01 00:09:21,2011-01-01 00:16:36,31111,31503,Member,7.0
4,233,2011-01-01 00:28:26,2011-01-01 00:32:19,31104,31106,Casual,4.0


In [17]:
bike_station2.dtypes


Duration                         int64
Start date              datetime64[ns]
End date                datetime64[ns]
Start station number             int64
End station number               int64
Member type                     object
dura_min                       float64
dtype: object

In [18]:
#MEMBERS FIX

In [19]:
bike_station2.loc[bike_station2['Member type'] == "Casual", 'Member cat'] = "1"
bike_station2.loc[bike_station2['Member type'] == "Member", 'Member cat'] = "2"
bike_station2.loc[bike_station2['Member type'] == "Unknown", 'Member cat'] = "1"

In [20]:
bike_station2.head(5)

,Duration,Start date,End date,Start station number,End station number,Member type,dura_min,Member cat
0,3548,2011-01-01 00:01:29,2011-01-01 01:00:37,31620,31620,Member,59.0,2
1,346,2011-01-01 00:02:46,2011-01-01 00:08:32,31105,31101,Casual,6.0,1
2,562,2011-01-01 00:06:13,2011-01-01 00:15:36,31400,31104,Member,9.0,2
3,434,2011-01-01 00:09:21,2011-01-01 00:16:36,31111,31503,Member,7.0,2
4,233,2011-01-01 00:28:26,2011-01-01 00:32:19,31104,31106,Casual,4.0,1


In [21]:
bike_station2["Member cat"].info

<bound method Series.info of 0          2
1          1
2          2
3          2
4          1
          ..
1226762    2
1226763    2
1226764    2
1226765    2
1226766    2
Name: Member cat, Length: 1226767, dtype: object>

In [22]:
#fix the hot encoding

print(ohe)

<class 'sklearn.preprocessing._encoders.OneHotEncoder'>


In [23]:
#DAY FIX

In [24]:
bike_station2.head()

,Duration,Start date,End date,Start station number,End station number,Member type,dura_min,Member cat
0,3548,2011-01-01 00:01:29,2011-01-01 01:00:37,31620,31620,Member,59.0,2
1,346,2011-01-01 00:02:46,2011-01-01 00:08:32,31105,31101,Casual,6.0,1
2,562,2011-01-01 00:06:13,2011-01-01 00:15:36,31400,31104,Member,9.0,2
3,434,2011-01-01 00:09:21,2011-01-01 00:16:36,31111,31503,Member,7.0,2
4,233,2011-01-01 00:28:26,2011-01-01 00:32:19,31104,31106,Casual,4.0,1


In [25]:
#TESTING AND SUCH

In [26]:
bike_station2.head()

,Duration,Start date,End date,Start station number,End station number,Member type,dura_min,Member cat
0,3548,2011-01-01 00:01:29,2011-01-01 01:00:37,31620,31620,Member,59.0,2
1,346,2011-01-01 00:02:46,2011-01-01 00:08:32,31105,31101,Casual,6.0,1
2,562,2011-01-01 00:06:13,2011-01-01 00:15:36,31400,31104,Member,9.0,2
3,434,2011-01-01 00:09:21,2011-01-01 00:16:36,31111,31503,Member,7.0,2
4,233,2011-01-01 00:28:26,2011-01-01 00:32:19,31104,31106,Casual,4.0,1


In [27]:
#x_data = bike_station2[["dura_min", "Start station number"]]
#y_data = new_cat[["Member cat new"]]


In [28]:
#display(x_data.head())
#display(y_data.head())

# Split Data 

In [30]:
#x_train, x_test, y_train, y_test = train_test_split(x_data, y_data ,test_size = 0.2, shuffle=False)
#x_train, x_test, y_train, y_test

In [32]:
# data normalisation with sklearn
#from sklearn.preprocessing import MinMaxScaler

# fit scaler on training data
#norm = MinMaxScaler().fit(x_train)

# transform training data
#X_train_norm = norm.transform(x_train)

# transform testing data
#X_test_norm = norm.transform(x_test)



In [34]:
# data normalisation with sklearn
from sklearn.preprocessing import MinMaxScaler

# fit scaler on training data
#norm = MinMaxScaler().fit(x_train)

# transform training data
#X_train_norm = norm.transform(x_train)

# transform testing data
#X_test_norm = norm.transform(x_test)



In [ ]:
#bike_station.isnull().any()

In [ ]:
#reg = LR().fit(x_train, y_train)

In [35]:
#y_predicted = reg.predict(x_test)
#y_predicted

In [36]:
#reg.score(y_predicted, y_test)

In [37]:
#display(y_predicted)
#display(y_test.head())


In [38]:
#reg.r2_score(y_test, y_predicted)

In [39]:
#reg.coef_

In [40]:
#reg.intercept_

In [41]:
#reg.predict #(np.array([[3, 5]])) ex.

In [42]:
#y_predicted = reg.predict(x_test)
#y_predicted

In [ ]:
#Supply and Demand analysis (based on bike stations).

#take the most used start station vs. the most used end station.

In [43]:
# Here I am creating a new dataframe after first selecting all the rows with the "Casual" and then grouping them by Hour and then counting it using .count()
# renaming the column to "Casual"
casual = bike_station[bike_station['Member type'] == 'Casual'].groupby(pd.Grouper(key = 'Start date', freq = 'H')).count().rename(columns={'Member type': 'Casual'}).reset_index()
casual = casual[['Start date', "Casual"]]
casual.head()

TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Index'